<a href="https://colab.research.google.com/github/saadatialirezam-create/glaucoma-detection/blob/main/01_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras import layers, models

In [2]:
from google.colab import drive
drive.mount('/content/drive')

train = np.load("/content/drive/MyDrive/Glaucoma_data/splits/train_split.npz")
test  = np.load("/content/drive/MyDrive/Glaucoma_data/splits/test_split.npz")

X_train = train["X"]
y_train = train["y"]
X_test  = test["X"]
y_test  = test["y"]

print("Train:", X_train.shape, y_train.shape, X_train.dtype)
print("Test :", X_test.shape,  y_test.shape,  X_test.dtype)

if X_train.ndim == 3:
    X_train = X_train[..., None]
if X_test.ndim == 3:
    X_test = X_test[..., None]

print("After channel fix - Train:", X_train.shape, "Test:", X_test.shape)

Mounted at /content/drive
Train: (1898, 224, 224) (1898,) uint8
Test : (475, 224, 224) (475,) uint8
After channel fix - Train: (1898, 224, 224, 1) Test: (475, 224, 224, 1)


In [3]:
SEED = 18
rng = np.random.default_rng(SEED)
idx = rng.permutation(len(y_train))
X_train = X_train[idx]
y_train = y_train[idx]

val_ratio = 0.2
val_size = int(len(y_train) * val_ratio)

X_val = X_train[:val_size]
y_val = y_train[:val_size]
X_tr  = X_train[val_size:]
y_tr  = y_train[val_size:]

print("Train split:", X_tr.shape, y_tr.shape)
print("Val split  :", X_val.shape, y_val.shape)

Train split: (1519, 224, 224, 1) (1519,)
Val split  : (379, 224, 224, 1) (379,)


In [4]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def to_rgb_and_preprocess(x, y):
    x = tf.cast(x, tf.float32)
    x = tf.cond(tf.reduce_max(x) <= 1.0, lambda: x * 255.0, lambda: x)
    x = tf.image.grayscale_to_rgb(x)    # -> (224,224,3)
    x = preprocess_input(x)             # ResNet50 preprocessing
    return x, y

train_ds = tf.data.Dataset.from_tensor_slices((X_tr, y_tr)).shuffle(4096, seed=SEED).map(to_rgb_and_preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val, y_val)).map(to_rgb_and_preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(to_rgb_and_preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [5]:
base = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
base.trainable = False

inputs = layers.Input(shape=(224, 224, 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
#x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         2,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [6]:
lr = 1e-4
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ]
)

# --- Fit ---
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6),
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=callbacks
)

base.trainable = True

for layer in base.layers[:-30]:
    layer.trainable = False

print("Trainable layers in base:", sum(l.trainable for l in base.layers), "/", len(base.layers))

fine_tune_lr = 1e-5
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=fine_tune_lr),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ]
)

fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 32s 383ms/step - accuracy: 0.5332 - loss: 0.7200 - precision: 0.3701 - recall: 0.4373 - val_accuracy: 0.6702 - val_loss: 0.6404 - val_precision: 0.6463 - val_recall: 0.3557 - learning_rate: 1.0000e-04
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.6672 - loss: 0.6237 - precision: 0.5335 - recall: 0.3138 - val_accuracy: 0.6834 - val_loss: 0.6258 - val_precision: 0.6559 - val_recall: 0.4094 - learning_rate: 1.0000e-04
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step - accuracy: 0.7178 - loss: 0.5878 - precision: 0.6227 - recall: 0.3918 - val_accuracy: 0.6992 - val_loss: 0.6143 - val_precision: 0.6882 - val_recall: 0.4295 - learning_rate: 1.0000e-04
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.7127 - loss: 0.5886 - precision: 0.6200 - recall: 0.3670 - val_accuracy: 0.7045 - val_loss: 0.6035 - val_precision: 0.6947 - val_recall: 0.4430 - learning_rate: 1.0000e-04
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 114ms/s

In [7]:
test_metrics = model.evaluate(test_ds, verbose=1)
print("Test metrics:", dict(zip(model.metrics_names, test_metrics)))

15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step - accuracy: 0.7568 - loss: 0.5192 - precision: 0.7864 - recall: 0.5316
Test metrics: {'loss': 0.4962839186191559, 'compile_metrics': 0.7726315855979919}
